In [ ]:
import geopandas as gpd
import stackstac
import numpy as np
import json
import rioxarray
import xarray as xr

import pandas as pd

# import ray
from tqdm import tqdm
import shapely
import datetime
import dask

from dask.distributed import Client
import dask.diagnostics

import functools


import matplotlib.pyplot as plt

import pathlib

import os
import folium


from utils.functions import (
    ingest_img,
    compute_indices,
)

from utils.config import RAW_DIR, INTERMEDIATE_DIR

pd.set_option("display.max_columns", None)

In [ ]:
# # set up the dask cluster
client = Client()
client

In [ ]:
from_year: int = 2022
to_year: int = 2023

## define AOI

In [ ]:
# load POC AOI
aoi: gpd.GeoDataFrame = (
    gpd.read_file(RAW_DIR / "peru_olmos_C5&C6.geojson")
    .pipe(lambda x: x.to_crs(crs=x.estimate_utm_crs()))
    # .assign(geometry=lambda x: x.buffer(274.32).envelope)  # 300 yards
    .to_crs(4326)
    .assign(geometry=lambda x: x.geometry.force_2d())
)

In [ ]:
# optional sense check
m = aoi.explore()
aoi.explore(m=m, color="red")


# add Google earth
m = m.add_child(
    folium.TileLayer(
        tiles="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
        name="Google Satellite",
        attr="Google",
        max_zoom=35,
        show=True,
    )
)

folium.LayerControl().add_to(m)

m

## Compute NDVI 

In [ ]:
aoi: gpd.GeoDataFrame = (
    aoi.drop(
        columns=["Description"],
        errors="ignore",
    )
    .dissolve()
    .assign(geometry=lambda x: x.envelope)
)

In [ ]:
# ingest the sentinel-2 image lazily
dds: xr.Dataset = ingest_img(
    aoi=aoi,
    from_year=from_year,
    to_year=to_year,
    collections_of_interest=["sentinel-2-l2a"],
    cloud_fraction_upper_bound_perc=100,
)

In [ ]:
# add a step to compute indices
dds_w_indices: xr.Dataset = dds.pipe(compute_indices)

In [ ]:
# round time by day to mosaic images from the same date
dds_w_indices["time"] = dds_w_indices["time"].dt.round("1D")

In [ ]:
# calculate daily mean NDVI
ndvi_daily: xr.Dataset = dds_w_indices["ndvi"].groupby("time").mean()

In [ ]:
# compute daily NDVI (takes long)
ndvi_daily: xr.Dataset = ndvi_daily.compute()

# Plot the daily NDVI as png

In [ ]:
dates: list[str] = ndvi_daily["time"].values

In [ ]:
for date in ndvi_daily["time"].values:
    fig, ax = plt.subplots(figsize=(10, 10))

    ndvi_daily.sel(time=date).plot(
        ax=ax,
        cmap="viridis",
        add_colorbar=False,
        vmin=0,
        vmax=1,
    )

    # add legend
    cbar = plt.colorbar(ax.collections[0], ax=ax, pad=0.01)
    cbar.set_label("NDVI", fontsize=15)
    cbar.ax.tick_params(labelsize=15)
    cbar.ax.set_yticklabels(cbar.ax.get_yticklabels(), fontsize=15)
    ax.set_xlabel("Longitude", fontsize=15)
    ax.set_ylabel("Latitude", fontsize=15)
    ax.tick_params(axis="both", which="major", labelsize=15)
    ax.set_title(f"NDVI {pd.to_datetime(date).strftime('%Y-%m-%d')}", fontsize=20)

    # axis equal
    ax.set_aspect("equal", adjustable="box")

    # plt.show()

    # save the figure
    plt.savefig(
        INTERMEDIATE_DIR / "ndvi_daily_{pd.to_datetime(date).strftime('%Y-%m-%d')}.png"
    )